In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/Jojo666/openai-to-z-challenge.git
pdf_dir = "openai-to-z-challenge/literature"


fatal: destination path 'openai-to-z-challenge' already exists and is not an empty directory.


In [ ]:
!pip uninstall -y fitz



Found existing installation: fitz 0.0.1.dev2
Uninstalling fitz-0.0.1.dev2:
  Successfully uninstalled fitz-0.0.1.dev2


In [ ]:
!pip install -U pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 80.4 MB/s eta 0:00:00


In [ ]:
import fitz  # This now works because pymupdf exposes itself as 'fitz'
import os

def extract_text_from_pdfs(pdf_folder):
    docs = []
    for file in os.listdir(pdf_folder):
        if file.endswith(".pdf"):
            path = os.path.join(pdf_folder, file)
            with fitz.open(path) as doc:
                text = "\n".join(page.get_text() for page in doc)
                docs.append({"filename": file, "text": text})
    return docs



In [ ]:
documents = extract_text_from_pdfs(pdf_dir)

**Split text into chunks for embedding**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = []
metas = []

for doc in documents:
    chunks = splitter.split_text(doc["text"])
    texts.extend(chunks)
    metas.extend([{"source": doc["filename"]}] * len(chunks))


**Embed and index chunks (use FAISS or ChromaDB)**

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"  # Replace with your actual key


In [ ]:
!pip install -U langchain-openai
from langchain_openai import OpenAIEmbeddings



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 21.8 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [ ]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 25.2 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(texts, embedding_model, metadatas=metas)


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4"), retriever=retriever)

response = qa_chain.run("Summarize findings about Amazonian archeological sites.")
print(response)


<ipython-input-17-317cb33b37da>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4"), retriever=retriever)
<ipython-input-17-317cb33b37da>:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run("Summarize findings about Amazonian archeological sites.")


The research has discovered various types of archaeological sites across Amazonia. The sample size includes 510 Amazonian Dark Earth sites (ADEs), 1,100 earthworks, and 422 other archaeological sites. Many earthwork sites are clearly visible in satellite imagery. In different regions of Amazonia, unreported earthworks, a fortified village, defensive and ceremonial sites, crowned mountains and megalithic structures, and riverine sites on floodplains were found. In the Peruvian Amazon, despite the low visibility of the archaeological sites and difficult living and working conditions, recent studies have suggested that the average Amazonian indigenous person at the end of the fifteenth century might have been part of an interconnected society rather than living in an isolated, autonomous village.


In [ ]:
response1 = qa_chain.run("Summarize links between Amazonian archeological sites and Brazil nuts.")
print(response1)

The research suggests a connection between the distribution of Brazil nut stands and Amazonian archaeological sites, such as Amazonian Dark Earth soils (ADE) and geoglyphs. The likelihood of finding Brazil nut stands showing signs of past human interaction increases from south-western to central and eastern Amazonia. The correlation is believed to be visible in the Brazil nut's demographic profiles observed across the Amazon Basin. It is common to find relatively clustered Brazil nut groves in central and eastern Amazonia, with densities ranging from 10 to 20 trees per hectare, interspersed with vast areas of forest. Finally, the research suggests that the Brazil nut may have reached certain areas through human influence, as long ago as 11,000 years.
